In [ ]:
# ==============================================================================
# notebooks/08_data_consolidation_and_feature_engineering.ipynb
# ==============================================================================

# # 08 - Data Consolidation and Feature Engineering
# This notebook brings together all the preprocessed data from previous steps:
# 1.  Processed Sentinel-2 vegetation indices (`woreda_monthly_vegetation_indices.csv`)
# 2.  Processed ERA5-Land, SRTM, and SMAP environmental data (`woreda_monthly_environmental_data.csv`)
# 3.  Prepared annual yield data (`woreda_annual_yield_data.csv`)
#
# It then performs additional feature engineering, such as creating lagged variables and seasonal aggregations,
# to build the final comprehensive dataset (`master_woreda_data.csv`) ready for machine learning modeling.

# ## 1. Load Project Setup and Libraries
# We'll load `pandas` for data manipulation and specify the directory where all processed data files are located.

import pandas as pd
import os
import numpy as np

print("Libraries loaded.")

# Define processed data directory
processed_data_dir = '../data/processed/'
os.makedirs(processed_data_dir, exist_ok=True)

print(f"Processed data directory: {processed_data_dir}")

# ## 2. Load Processed Data Files
# Load the vegetation indices, environmental data, and yield data CSV files saved from previous notebooks.

try:
    df_vi = pd.read_csv(os.path.join(processed_data_dir, 'woreda_monthly_vegetation_indices.csv'))
    print(f"Loaded vegetation indices: {df_vi.shape[0]} records.")
    print(df_vi.head())

    df_env = pd.read_csv(os.path.join(processed_data_dir, 'woreda_monthly_environmental_data.csv'))
    print(f"\nLoaded environmental data: {df_env.shape[0]} records.")
    print(df_env.head())

    df_yield = pd.read_csv(os.path.join(processed_data_dir, 'woreda_annual_yield_data.csv'))
    print(f"\nLoaded annual yield data: {df_yield.shape[0]} records.")
    print(df_yield.head())

except FileNotFoundError as e:
    print(f"Error loading data: {e}. Please ensure you have run notebooks 05, 06, and 07 to generate these files.")
    df_vi, df_env, df_yield = None, None, None # Set to None to prevent further execution

# ## 3. Consolidate Monthly Data
# Merge the vegetation indices and environmental data into a single monthly DataFrame. This forms the base for time-series feature engineering.

if df_vi is not None and df_env is not None:
    # Merge vegetation indices and environmental data by woreda_id, year, and month
    df_monthly_features = pd.merge(df_vi, df_env,
                                   on=['woreda_id', 'woreda_name', 'year', 'month'],
                                   how='outer') # Use outer to keep all available records

    # Sort for correct lagged calculations
    df_monthly_features = df_monthly_features.sort_values(by=['woreda_id', 'year', 'month']).reset_index(drop=True)

    print(f"\nConsolidated monthly features: {df_monthly_features.shape[0]} records.")
    print(df_monthly_features.head())
else:
    print("Skipping monthly data consolidation due to missing input data.")

# ## 4. Feature Engineering: Lagged Variables and Seasonal Aggregations
# This section generates additional features from the monthly data to capture temporal patterns that might influence annual yield.
# Common techniques include:
# -   **Lagged features:** Values from previous months/seasons (e.g., NDVI 3 months prior).
# -   **Seasonal statistics:** Averages or sums over specific growing seasons (e.g., average precipitation during the main rainy season).

if 'df_monthly_features' in locals() and df_monthly_features is not None:
    # --- Create Lagged Features (Example: Lagging by 1 month) ---
    # Identify columns to lag (exclude identifiers and static features like elevation)
    cols_to_lag = [
        'avg_ndvi', 'avg_savi',
        'era5_total_precipitation', 'era5_temperature_2m',
        'era5_surface_pressure', 'era5_soil_temperature_level_1', 'era5_soil_volume_water_content_level_1',
        'smap_sm_surface_pressure', 'smap_sm_rootzone_pressure'
    ]

    # Create a unique time identifier for sorting within groups
    df_monthly_features['date_id'] = df_monthly_features['year'] * 100 + df_monthly_features['month']
    df_monthly_features = df_monthly_features.sort_values(by=['woreda_id', 'date_id'])

    for col in cols_to_lag:
        for lag in [1, 2, 3]: # Example lags: 1, 2, and 3 months prior
            df_monthly_features[f'{col}_lag{lag}'] = df_monthly_features.groupby('woreda_id')[col].shift(lag)

    print(f"\nMonthly features with lagged variables: {df_monthly_features.shape[0]} records, {df_monthly_features.shape[1]} columns.")
    print(df_monthly_features.head())

    # --- Create Seasonal Aggregations ---
    # Define seasons based on coffee growing cycle (example for Ethiopia/Sidama)
    # Main rainy season (Kremt): June, July, August, September
    # Dry season (Bega): October, November, December, January, February
    # Short rainy season (Belg): March, April, May

    # For yield prediction for a given year Y, features often come from year Y and Y-1.
    # Example: Average precipitation in the previous growing season.

    # Aggregate monthly features to annual level for modeling against annual yield
    # We'll aggregate features for the year *before* the yield year and potentially early months of the yield year itself.
    # This example aggregates all monthly features to a yearly mean for the current year.
    # You will likely need to refine these aggregations based on phenology and domain knowledge.

    # Group by woreda_id and year, then aggregate relevant features
    # Exclude identifiers and columns not suitable for mean aggregation
    agg_cols = [col for col in df_monthly_features.columns if col not in ['woreda_name', 'month', 'date_id']]

    # Ensure avg_elevation is not included in monthly aggregation, as it's static
    if 'avg_elevation' in agg_cols:
        agg_cols.remove('avg_elevation')

    # For simplicity, let's take the mean of monthly values for each year.
    # For better results, you'd define specific growing seasons and aggregate accordingly.
    df_annual_features = df_monthly_features.groupby(['woreda_id', 'year'])[agg_cols].mean().reset_index()

    # If avg_elevation was removed, re-add it from df_env or df_monthly_features (it's static per woreda)
    # A more robust way: merge elevation at the end or ensure it's not part of the monthly aggregation
    # For now, let's assume `avg_elevation` is handled. If it's missing, you can re-merge it.
    if 'avg_elevation' in df_monthly_features.columns:
        # Take the first non-null elevation per woreda_id
        woreda_elevation = df_monthly_features[['woreda_id', 'avg_elevation']].drop_duplicates().set_index('woreda_id')
        df_annual_features = df_annual_features.merge(woreda_elevation, on='woreda_id', how='left')

    print(f"\nAnnual features (after aggregation): {df_annual_features.shape[0]} records.")
    print(df_annual_features.head())
else:
    print("Skipping feature engineering due to missing consolidated monthly data.")

# ## 5. Final Data Consolidation (Features + Yield)
# Merge the engineered features with the annual yield data. This is the final dataset used for training and evaluating the yield prediction model.

if 'df_annual_features' in locals() and df_annual_features is not None and df_yield is not None:
    # Merge features with yield data
    # The 'annual_yield_quintals_ha' from df_yield will be the target variable
    df_master = pd.merge(df_annual_features, df_yield,
                         on=['woreda_id', 'year'],
                         how='left') # Use left join to keep all features and add yield if available

    # Add woreda name back for convenience if it was lost during aggregation
    if 'woreda_name' not in df_master.columns:
        df_woreda_names = df_monthly_features[['woreda_id', 'woreda_name']].drop_duplicates()
        df_master = pd.merge(df_master, df_woreda_names, on='woreda_id', how='left')

    # Reorder columns to have identifiers and target first
    cols = ['woreda_id', 'woreda_name', 'year', 'annual_yield_quintals_ha'] + \
           [col for col in df_master.columns if col not in ['woreda_id', 'woreda_name', 'year', 'annual_yield_quintals_ha']]
    df_master = df_master[cols]

    print(f"\nFinal master dataset shape: {df_master.shape}")
    print(df_master.head())

    # Display number of rows with missing yield data (these rows can be used for prediction)
    missing_yield_rows = df_master['annual_yield_quintals_ha'].isna().sum()
    print(f"Number of rows with missing yield data (for potential prediction): {missing_yield_rows}")

    # Save the master dataset
    master_data_path = os.path.join(processed_data_dir, 'master_woreda_data.csv')
    df_master.to_csv(master_data_path, index=False)
    print(f"✅ Master dataset saved to {master_data_path}")
else:
    print("Skipping final data consolidation due to missing input data.")
